In [28]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn as nn
import zipfile
import os

In [29]:
class SentimentBERT(nn.Module):

    class_interpretation = {
        0: "neutral",
        1: "positive",
        2: "negative"
    }

    def __init__(self, load_dir):
        super(SentimentBERT, self).__init__()
        self.bert = AutoModelForSequenceClassification.from_pretrained(load_dir)
        self.tokenizer = AutoTokenizer.from_pretrained(load_dir)

    def forward(self, text):
        inputs = self.tokenizer(text, return_tensors="pt")
        with torch.no_grad():
            logits = self.bert(**inputs).logits
        probabilities = torch.softmax(logits, dim=-1)

        result = {"class": SentimentBERT.class_interpretation[probabilities.argmax().item()],
                   "probabilities": probabilities }
        
        return result

In [30]:
zip_file_path = 'model.zip'  
output_directory = './model'   



def unzip_file(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zipf:
        zipf.extractall(extract_to)
        print(f'Файлы из "{zip_path}" были успешно извлечены в "{extract_to}".')

os.makedirs(output_directory, exist_ok=True)
unzip_file(zip_file_path, output_directory)

model = SentimentBERT(output_directory)

Файлы из "model.zip" были успешно извлечены в "./model".


In [31]:
text = '''Может кто-то знает, подлесных требует ото всех членов в команде сетевое взаимодействие? Или достаточно чтобы в самом проекте было сетевое взаимодействие?'''
model(text)

{'class': 'neutral', 'probabilities': tensor([[0.9277, 0.0261, 0.0463]])}

In [15]:
text = '''я тебя ненавижу'''
model(text)

{'class': 'negative', 'probabilities': tensor([[0.0707, 0.0090, 0.9203]])}

In [16]:
text = '''я тебя ненавижу (почти)'''
model(text)

{'class': 'negative', 'probabilities': tensor([[0.2157, 0.0133, 0.7709]])}

In [17]:
text = '''я возможно тебя ненавижу (почти)'''
model(text)

{'class': 'negative', 'probabilities': tensor([[0.3646, 0.0142, 0.6212]])}

In [19]:
text = '''я возможно тебя ненавижу (почти), но все равно ты хороший'''
model(text)

{'class': 'neutral', 'probabilities': tensor([[0.5768, 0.1619, 0.2612]])}

In [20]:
text = '''Путин победил на выборах президента'''
model(text)

{'class': 'neutral', 'probabilities': tensor([[0.8931, 0.0916, 0.0153]])}

In [21]:
text = '''Ура! Путин победил на выборах президента'''
model(text)

{'class': 'positive', 'probabilities': tensor([[0.0467, 0.9508, 0.0025]])}